<a href="https://colab.research.google.com/github/JoosepAlbre/IDS_Project/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>